In [198]:
import pandas as pd
from string import punctuation
import re
import numpy as np
from setup import *
import glob
import os
import pickle
import joblib
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from keras.preprocessing import sequence


keep_reach = 50



# The Process

## [I. Data Acquisition](#data_acquisition_link)

## [II. Preprocessing](#preprocessing_link)

## [III. Setup The Model](#setup_link)


## Model Setup
1. Support Vector Machine Regression
1. test-train split

## Train The Model, Tune Hyperparameters
1. Gridsearch on some parameters (maybe include how much reach to give to model)
1. Save all of the models, compare outputs, etc
1. Cross validate

## Test Model
1. Test each model on the testing set, look for best results. 

## Save Best Model 
1. Use pickle to save the best model for future use. 

<a id='data_acquisition_link'></a>
# Data Acquisition
1. Check if Files have been collected already
1. Use Glob to collect all of the files in the Data Folder
1. Save to single file for later use

In [8]:

if os.path.isfile('All_Files.csv'):
    print('Using old dataframe')
    df = pd.read_csv('All_Files.csv').

else:
    print('Making new dataframe')
    file_list = glob.glob("./data/Facebook Insights*")

    df_list = []

    for file in file_list:
        df_list.append(pd.read_csv(file,encoding='iso-8859-1'))

    # for df in df_list:
    #     display(df)

    df = pd.concat(df_list).reset_index()
    df.to_csv('All_Files.csv')
display(df)

Using old file


,Unnamed: 0,index,Post ID,Permalink,Post Message,Type,Countries,Languages,Posted,Audience Targeting,...,Lifetime Organic views to 95%,Lifetime Organic views to 95%.1,Lifetime Paid views to 95%,Lifetime Paid views to 95%.1,Lifetime Organic Video Views,Lifetime Organic Video Views.1,Lifetime Paid Video Views,Lifetime Paid Video Views.1,Lifetime Average time video viewed,Lifetime Video length
0,0,1,171162119680585_859052127558244,https://www.facebook.com/TarletonCOGS/posts/85...,Tomorrow is the COGS fall open house at Tarlet...,Link,nan,nan,9/30/16 4:26 PM,,...,0,0,0,0,0,0,0,0,0,0
1,1,2,171162119680585_859037424226381,https://www.facebook.com/TarletonCOGS/posts/85...,It's not too late to register for Saturday's O...,Link,nan,nan,9/29/16 4:11 PM,,...,0,0,0,0,0,0,0,0,0,0
2,2,3,171162119680585_859081790888611,https://www.facebook.com/TarletonCOGS/posts/85...,Tarleton's Fall Job Fair is being held October...,Link,nan,nan,9/29/16 6:19 AM,,...,0,0,0,0,0,0,0,0,0,0
3,3,4,171162119680585_859035847559872,https://www.facebook.com/TarletonCOGS/posts/85...,Questions about the #GRE? Attend this Saturda...,Link,nan,nan,9/28/16 4:07 PM,,...,0,0,0,0,0,0,0,0,0,0
4,4,5,171162119680585_859091770887613,https://www.facebook.com/TarletonCOGS/posts/85...,The deadline to register for the #TAMUSPathway...,Link,nan,nan,9/28/16 9:40 AM,,...,0,0,0,0,0,0,0,0,0,0
5,5,6,171162119680585_859074730889317,https://www.facebook.com/TarletonCOGS/posts/85...,Access the services of The Writing Center at ...,Link,nan,nan,9/28/16 6:02 AM,,...,0,0,0,0,0,0,0,0,0,0
6,6,7,171162119680585_859035654226558,https://www.facebook.com/TarletonCOGS/posts/85...,Learn more about graduate programs at Tarleton...,Link,nan,nan,9/27/16 4:07 PM,,...,0,0,0,0,0,0,0,0,0,0
7,7,8,171162119680585_859058130890977,https://www.facebook.com/TarletonCOGS/posts/85...,Looking to further your career as an education...,Photo,nan,nan,9/27/16 6:43 AM,,...,0,0,0,0,0,0,0,0,0,0
8,8,9,171162119680585_859028140893976,https://www.facebook.com/TarletonCOGS/posts/85...,Did you miss Tarleton on Tour on Saturday? Yo...,Link,nan,nan,9/26/16 4:53 PM,,...,0,0,0,0,0,0,0,0,0,0
9,9,10,171162119680585_859018744228249,https://www.facebook.com/TarletonCOGS/posts/85...,Thanks to all who came by the Arlington Conven...,Photo,nan,nan,9/26/16 7:17 AM,,...,0,0,0,0,0,0,0,0,0,0


<a id='preprocessing_link'></a>
# Preprocessing

1. Collect the columns for reach, engagement
1. Dropna, since they will not contribute to finding good posts
1. Create proportion column
1. Tokenize
1. Tf-Idf
1. Save the Tf-Idf using pickle
1. Condense the preprocessing step into a function ** TO DO STILL **


Use This next cell to restart the fresh df, to check changes in the preprocessing

In [230]:
overwrite = True

if os.path.isfile('vectorizer.pkl') and overwrite is False:
    print('Using old pickle')
    df = pd.read_csv('All_Files.csv')
    X_all = pd.read_csv('all_X.csv',header=None, index_col = 0)[1]
    y_all = pd.read_csv('all_y.csv',header=None, index_col = 0,skiprows = 1)[1]
    tfidf_vectorizer = joblib.load('vectorizer.pkl')

else:
    df = pd.read_csv('All_Files.csv')
    df = df[['Post Message', 'Type', 'Posted', 'Lifetime Post Total Reach','Lifetime Engaged Users']]
    df.columns = ['Post', 'Type','Date','Reach','Engaged']
    df.dropna(inplace=True)
    df = pd.merge(df.groupby('Post')['Engaged'].apply(lambda p: max(p)).reset_index(),df[['Post', 'Type', 'Date', 'Reach', 'Engaged']], on = ['Post','Engaged'], how = 'inner')
    df.drop_duplicates('Post',inplace=True)
    df = df.loc[df['Reach']>=keep_reach].reset_index(drop=True)
    df['Prop'] = df['Engaged']/df['Reach']
    tokenized_df = df
    tokenized_df['Sentences'] = tokenized_df['Post'].transform(lambda x: sent_tokenize(str(x)))
    tokenized_df['Words'] = tokenized_df['Post'].transform(lambda x: ' '.join([WordNetLemmatizer().lemmatize(y.lower()) for y in word_tokenize(x) if y.isalpha()]))
    tokenized_df['Words'] = tokenized_df['Words'].transform(lambda x: ' '.join([y for y in word_tokenize(x) if y not in stopwords.words('english')]))
    tokenized_df = tokenized_df.loc[tokenized_df['Words'] != 'http']
    X_all = tokenized_df['Words']
    scaler = MinMaxScaler()
    y_all = scaler.fit_transform(tokenized_df['Prop'].values.reshape(-1,1)).ravel()
    X_all.to_csv('all_X.csv')
    pd.DataFrame(y_all).to_csv('all_y.csv') 
    tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', min_df = 5, max_df = 0.25, ngram_range = (1,2)) #words must be in at least 5 posts and no more than 25% of all posts
    tfidf_vectorizer.fit(X_all)
    joblib.dump(Tfidf_vectorizer, 'vectorizer.pkl')

transformed_X = Tfidf_vectorizer.transform(X_all)

In [231]:
y_all.shape

(779,)

<a id='setup_link'></a>
### Setup the Model
1. Load the Model from scikit learn

In [247]:
from sklearn.metrics import mean_absolute_error
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y_all, test_size=0.25)

model = SVR(epsilon = 0.001)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))


0.0780435631651


In [248]:
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_test, y_pred)))

0.112755861993
